<a href="https://colab.research.google.com/github/BecomeAllan/RNN/blob/main/Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratório

## Implementações com RNNs

### Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import numpy as np
import os
import time
import re

!pip install unidecode
from unidecode import unidecode

tf.config.list_physical_devices('GPU')

## Tratamento de dados

In [ ]:
# Baixar dataset
!gdown --id 15FhPHu7Hx6ul_k-EEBZwzpUWznK0gBR3


In [ ]:
# Data1
df1 = pd.read_csv("portuguese-poems.csv", encoding='UTF-8')

df1 = df1.dropna()


df1.head()

In [454]:
df1['Title'] = df1['Title'].apply(unidecode).str.lower()
df1['Content'] = df1['Content'].apply(unidecode).str.lower()

# Creado os dados
data = pd.DataFrame()

data['Econder_inputs'] = df1['Title']
#.apply(lambda row: re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", row)).apply(vectorize_string)
data['Decoder_inputs'] = df1['Content'].apply(lambda row: "<BOS> " + row[:-1])
#.apply(lambda row: re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", row))
data['Decoder_targets'] = df1['Content'].apply(lambda row: row[1:] + " <EOS>")
#.apply(lambda row: re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", row)).apply(lambda row: vectorize_string(row[1:]))

In [456]:
# Treina os Tokens
tokenizer = Tokenizer(char_level=True, lower=False)

tokenizer.fit_on_texts(data['Econder_inputs'])
tokenizer.fit_on_texts(data['Decoder_targets'])
tokenizer.fit_on_texts(data['Decoder_inputs'])

dictionary = tokenizer.word_index

k = tokenizer.texts_to_sequences(data['Econder_inputs'][0])
k_text = tokenizer.sequences_to_texts(k)

print(dictionary)
print(f'O input: {data["Econder_inputs"][0]}')
print(f'O Token do input: {k}')
print(f'O decode do Token do input: {k_text}')


data['Econder_inputs'] = tokenizer.texts_to_sequences(data['Econder_inputs'])

data['Decoder_inputs'] = tokenizer.texts_to_sequences(data['Decoder_inputs'])

data['Decoder_targets'] = tokenizer.texts_to_sequences(data['Decoder_targets'])

list

In [419]:
MAX_LENGTH_TITLE = 100
MAX_LENGTH_POEM = 250

data_Econder_inputs = pad_sequences(data['Econder_inputs'], maxlen=MAX_LENGTH_TITLE, padding='post', truncating='post')

data_Decoder_inputs = pad_sequences(data['Decoder_inputs'], maxlen=MAX_LENGTH_POEM, padding='post', truncating='post')

data_Decoder_targets = pad_sequences(data['Decoder_targets'], maxlen=MAX_LENGTH_POEM, padding='post', truncating='post')


Encoder_Train, Encoder_Test, _, _ = train_test_split(data_Econder_inputs, data_Decoder_inputs, test_size = 0.2, random_state=5)
Decoder_Train, Decoder_Test, Target_Train, Target_Test  =  train_test_split(data_Decoder_inputs, data_Decoder_targets, test_size = 0.2, random_state=5)

Target_Train = keras.utils.to_categorical(Target_Train, num_classes=len(dictionary))
Target_Test = keras.utils.to_categorical(Target_Test, num_classes=len(dictionary))


In [ ]:
# Length of the vocabulary in chars
vocab_size = len(dictionary)

# The embedding dimension
embedding_dim = 76

# Number of RNN units
rnn_units = 50


## Encoder
encode_input = keras.Input(shape=(None,), name="title")
encode_features = layers.Embedding(vocab_size, embedding_dim)(encode_input) 
encoder = layers.LSTM(rnn_units, return_state=True, name = 'encode')
encode_output, state_h, state_c = encoder(encode_features)

# Estado da celula 
encoder_state = [state_h, state_c]



#layers.CuDNNLSTM

## Decoder
decode_input = keras.Input(shape=(None,), name="content")
decode_features = layers.Embedding(vocab_size, embedding_dim)(decode_input)
decode = layers.LSTM(rnn_units, return_state=True, return_sequences=True, name = 'decode')
decode_out, _, _ = decode(decode_features, initial_state = encoder_state)
decoder_outputs = layers.TimeDistributed(layers.Dense(vocab_size, activation="softmax"))(decode_out)

# Estado da celula 


model = keras.Model([encode_input, decode_input], decoder_outputs)
model.summary()

#keras.backend.clear_session()

#keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss = "categorical_crossentropy",
    metrics = keras.metrics.categorical_accuracy
)

#class CustomCallback(keras.callbacks.Callback):
#  def on_epoch_end(self, epoch, logs=None):
#    if 0 == epoch%1: 
#        print(f'Época: {epoch} \n output:{logs["predict"]}')
        

model.fit([Encoder_Train, Decoder_Train], Target_Train,
          batch_size = 3000,
          epochs=2, verbose=2)

pred = model.predict([Encoder_Test[0:2], Decoder_Test[0:2]])

# Predições 

pred = np.argmax(pred[0], axis=1)

 
enc = tokenizer.sequences_to_texts([ [idx] for idx in Encoder_Test[0].tolist()])
res = tokenizer.sequences_to_texts([ [idx] for idx in pred.tolist()])

print(f'Encode: {"".join(enc)}')
print("".join(res))
